In [ ]:
print('Setup complete.')

# Lab 6: Build a Prompt Library

## Learning Objectives
- Create a reusable library of battle-tested prompts
- Implement prompt templates with variable substitution
- Build validation and testing frameworks for prompts
- Design prompt versioning and management systems

## Lab Overview
You'll build a comprehensive prompt library system that stores, manages, and validates prompts for common tasks. This becomes your "prompt arsenal" for future projects.

**Estimated Time:** 60 minutes

## Your Mission
Create a production-ready prompt library with templates, validation, and performance tracking.

In [ ]:
# Install required packages
!pip install jinja2 pydantic rich jsonlines pyyaml tiktoken

In [ ]:
# ================================
# 🔐 Cell 1 — Load secrets (Colab) + pricing + token utils
# ================================
import os, time, csv
from typing import Optional, Dict
import tiktoken


from google.colab import userdata


ASKSAGE_API_KEY = userdata.get("ASKSAGE_API_KEY")
ASKSAGE_BASE_URL = userdata.get("ASKSAGE_BASE_URL")
ASKSAGE_EMAIL = userdata.get("ASKSAGE_EMAIL")



assert ASKSAGE_API_KEY, "ASKSAGE_API_KEY not provided."
assert ASKSAGE_EMAIL, "ASKSAGE_EMAIL not provided."


print("✓ Secrets loaded")
print("  • EMAIL:", ASKSAGE_EMAIL)
print("  • BASE URL:", ASKSAGE_BASE_URL or "(default)")


In [ ]:
import sys
sys.path.append('../../../')  # Adjust path to reach bootcamp_common

from bootcamp_common.ask_sage import AskSageClient

# Initialize AskSage client
client = AskSageClient(
    api_key=ASKSAGE_API_KEY,
    base_url=ASKSAGE_BASE_URL
)

print("✓ AskSage client initialized")

## Task 1: Design Prompt Template System

**TODO:** Create a flexible prompt template system with validation.

In [ ]:
from pydantic import BaseModel, ValidationError, Field, validator
from jinja2 import Template
from pathlib import Path
import yaml
from datetime import datetime
from dataclasses import dataclass, asdict

class PromptTemplate(BaseModel):
    """Structured prompt template with validation"""
    id: str
    name: str
    description: str
    category: str
    template: str
    variables: List[str]
    system_message: Optional[str] = None
    examples: List[Dict[str, str]] = []
    tags: List[str] = []
    version: str = "1.0"
    author: str = "bootcamp"
    created_at: str = None
    performance_metrics: Dict[str, float] = {}

@dataclass
class PromptExecution:
    """Record of prompt execution with metadata"""
    template_id: str
    variables: Dict[str, Any]
    rendered_prompt: str
    response: str
    success: bool
    response_time: float
    token_count: int
    cost: float
    timestamp: str

class PromptLibrary:
    """Main prompt library management system"""
    
    def __init__(self, library_path: str = "prompt_library"):
        self.library_path = Path(library_path)
        self.library_path.mkdir(exist_ok=True)
        self.templates: Dict[str, PromptTemplate] = {}
        self.execution_log: List[PromptExecution] = []
        self._load_templates()
        
    def _load_templates(self):
        """Load templates from YAML files"""
        for yaml_file in self.library_path.glob("*.yaml"):
            try:
                with open(yaml_file, 'r') as f:
                    template_data = yaml.safe_load(f)
                template = PromptTemplate(**template_data)
                self.templates[template.id] = template
            except Exception as e:
                console.print(f"[red]Error loading {yaml_file}: {e}[/red]")
        
        # Create starter templates if none exist
        if not self.templates:
            self._create_starter_templates()
    
    def _create_starter_templates(self):
        """Create basic starter templates"""
        starter_templates = [
            {
                "id": "text_summarizer",
                "name": "Text Summarizer",
                "description": "Summarize text with specified length and style",
                "category": "text_processing",
                "template": """Summarize the following text in {{ max_words }} words or less.
Style: {{ style }}
Focus on: {{ focus_areas }}

Text:
{{ text }}

Summary:""",
                "variables": ["text", "max_words", "style", "focus_areas"],
                "examples": [
                    {
                        "text": "Long article about AI trends...",
                        "max_words": "50",
                        "style": "bullet points",
                        "focus_areas": "key developments and future outlook"
                    }
                ],
                "tags": ["summarization", "text_processing"],
                "created_at": datetime.now().isoformat()
            },
            {
                "id": "email_classifier",
                "name": "Email Classifier",
                "description": "Classify emails by priority and category",
                "category": "classification",
                "template": """Classify this email:

Email: {{ email_content }}

Priority: [urgent/normal/low]
Category: [work/personal/spam/newsletter]
Reasoning:""",
                "variables": ["email_content"],
                "tags": ["classification", "email"],
                "created_at": datetime.now().isoformat()
            }
        ]
        
        for template_data in starter_templates:
            template = PromptTemplate(**template_data)
            self.templates[template.id] = template
            self.save_template(template)
    
    def add_template(self, template_data: Dict) -> str:
        """Add new template to library"""
        try:
            template_data["created_at"] = datetime.now().isoformat()
            template = PromptTemplate(**template_data)
            self.templates[template.id] = template
            self.save_template(template)
            return template.id
        except ValidationError as e:
            console.print(f"[red]Template validation error: {e}[/red]")
            return None
    
    def get_template(self, template_id: str) -> Optional[PromptTemplate]:
        """Retrieve template by ID"""
        return self.templates.get(template_id)
    
    def render_prompt(self, template_id: str, variables: Dict[str, Any]) -> str:
        """Render template with variables using Jinja2"""
        template = self.get_template(template_id)
        if not template:
            raise ValueError(f"Template {template_id} not found")
        
        # Validate required variables
        missing_vars = set(template.variables) - set(variables.keys())
        if missing_vars:
            raise ValueError(f"Missing required variables: {missing_vars}")
        
        jinja_template = Template(template.template)
        return jinja_template.render(**variables)
    
    # TODO: Implement validate_template() function
    # This function should:
    # - Check required fields are present
    # - Validate Jinja2 template syntax
    # - Verify variable consistency between template and variables list
    # - Validate examples match template variables
    # - Return list of validation errors
    
    def save_template(self, template: PromptTemplate):
        """Save template to disk as YAML"""
        file_path = self.library_path / f"{template.id}.yaml"
        template_dict = template.dict()
        with open(file_path, 'w') as f:
            yaml.dump(template_dict, f, default_flow_style=False)
    
    # TODO: Implement execute_template() function
    # This function should:
    # - Render the template with provided variables
    # - Use client.query() to get AI response
    # - Track execution time, tokens, and cost
    # - Store execution record in self.execution_log
    # - Return PromptExecution object

# Initialize library
library = PromptLibrary()
console.print(f"📚 Prompt library initialized with {len(library.templates)} templates")
console.print("⚠️ TODO: Complete validate_template() and execute_template() functions")

## Task 2: Build Template Browser and Search

**TODO:** Create tools to browse, search, and discover templates.

In [ ]:
# TODO: Build Template Browser and Search
#
# Your task:
# 1. Create browse_templates() function with rich table display
# 2. Implement search_templates() with multi-field search
# 3. Build show_template_details() for detailed template view
# 4. Create create_template_tree() for hierarchical category view
# 5. Add filtering by category, tags, and performance metrics
#
# Features to implement:
# - Rich table display with ID, name, category, variables, tags
# - Search across name, description, tags, and content
# - Detailed template panels with examples and metadata
# - Tree view organized by categories
# - Performance-based recommendations
# - Fuzzy matching for better search results
#
# Integration with AskSage:
# - Preview template execution with sample data
# - Show estimated costs for template usage
# - Display performance metrics from previous runs

print("⚠️ TODO: Implement template browser and search functionality")

## Task 3: Create Template Testing Framework

**TODO:** Build a system to test and validate prompt templates.

In [ ]:
class PromptTester:
    """Test prompt templates for quality and performance"""
    
    def __init__(self, library: PromptLibrary):
        self.library = library
        self.setup_client()
    
    def setup_client(self):
        """Setup API client"""
        self.has_api = os.getenv('OPENAI_API_KEY') is not None
        if self.has_api:
            import openai
            self.client = openai.OpenAI()
            console.print("✅ API client configured")
        else:
            console.print("💡 Using mock responses for testing")
    
    def test_template(self, template_id: str, test_variables: Dict[str, Any]) -> PromptExecution:
        """TODO: Test a template with given variables"""
        template = self.library.get_template(template_id)
        if not template:
            raise ValueError(f"Template {template_id} not found")
        
        # TODO: Render prompt with test variables
        # TODO: Execute prompt via API or mock
        # TODO: Measure performance metrics
        # TODO: Return execution record
        
        start_time = time.time()
        
        rendered_prompt = self.library.render_prompt(template_id, test_variables)
        
        if self.has_api:
            # TODO: Make actual API call
            response = "API response here"
            token_count = 50
        else:
            # Mock response
            response = f"Mock response for {template_id} with variables: {test_variables}"
            token_count = len(response.split())
        
        execution = PromptExecution(
            template_id=template_id,
            variables=test_variables,
            rendered_prompt=rendered_prompt,
            response=response,
            success=True,
            response_time=time.time() - start_time,
            token_count=token_count,
            timestamp=datetime.now().isoformat()
        )
        
        self.library.execution_log.append(execution)
        return execution
    
    def run_template_tests(self, template_id: str) -> List[PromptExecution]:
        """TODO: Run all example tests for a template"""
        template = self.library.get_template(template_id)
        if not template:
            raise ValueError(f"Template {template_id} not found")
        
        results = []
        
        # TODO: Run tests using template examples
        # TODO: Validate outputs meet expectations
        # TODO: Generate test report
        
        for example in template.examples:
            try:
                result = self.test_template(template_id, example)
                results.append(result)
            except Exception as e:
                console.print(f"[red]Test failed: {e}[/red]")
        
        return results
    
    def benchmark_template(self, template_id: str, iterations: int = 5) -> Dict[str, float]:
        """TODO: Benchmark template performance"""
        # TODO: Run template multiple times
        # TODO: Calculate average metrics
        # TODO: Return performance statistics
        
        metrics = {
            "avg_response_time": 0.0,
            "avg_token_count": 0.0,
            "success_rate": 0.0,
            "consistency_score": 0.0
        }
        
        return metrics
    
    def validate_template_quality(self, template_id: str) -> Dict[str, Any]:
        """TODO: Assess template quality across multiple dimensions"""
        quality_report = {
            "clarity_score": 0.0,        # How clear are the instructions?
            "completeness_score": 0.0,   # Are all necessary elements present?
            "consistency_score": 0.0,    # Does it produce consistent outputs?
            "efficiency_score": 0.0,     # Token usage efficiency
            "overall_score": 0.0,
            "recommendations": []
        }
        
        # TODO: Implement quality scoring algorithms
        # TODO: Analyze template structure and content
        # TODO: Generate improvement recommendations
        
        return quality_report

# Initialize tester
tester = PromptTester(library)
print("🧪 Template tester ready!")
print("⚠️ TODO: Complete the testing and validation methods above")

## Task 4: Demonstrate Library Usage

**TODO:** Show how to use your prompt library system.

In [ ]:
# TODO: Demonstrate library usage

console.print("\n📚 [bold blue]Prompt Library Demonstration[/bold blue]")

# 1. Browse available templates
console.print("\n[yellow]1. Available Templates:[/yellow]")
console.print(browser.list_templates())

# 2. Show template details
console.print("\n[yellow]2. Template Details:[/yellow]")
# TODO: Show details for an existing template
if "text_summarizer" in library.templates:
    browser.show_template_details("text_summarizer")

# 3. Test a template
console.print("\n[yellow]3. Testing Template:[/yellow]")
# TODO: Test the template with sample data
test_vars = {
    "text": "Artificial intelligence is rapidly transforming industries...",
    "max_words": "30",
    "style": "professional",
    "focus_areas": "business impact"
}

try:
    result = tester.test_template("text_summarizer", test_vars)
    console.print(f"[green]✅ Test successful![/green]")
    console.print(Panel(result.response, title="Generated Response", border_style="green"))
    console.print(f"Response time: {result.response_time:.2f}s, Tokens: {result.token_count}")
except Exception as e:
    console.print(f"[red]❌ Test failed: {e}[/red]")

print("\n📊 Library demonstration complete!")
print("⚠️ TODO: Add more templates and improve the demonstration")

## Task 5: Create Your Custom Templates

**TODO:** Design and add 3-5 custom templates for different use cases.

In [ ]:
# TODO: Create custom templates for your prompt library

def create_custom_templates():
    """TODO: Design and add custom prompt templates"""
    
    # TODO: Create templates for:
    # 1. Code review and feedback
    # 2. Meeting notes extraction
    # 3. Creative story generation
    # 4. Technical documentation
    # 5. Email response drafting
    
    custom_templates = [
        # TODO: Define your custom templates here
        # Use the same structure as the starter templates
        {
            "id": "code_reviewer",
            "name": "Code Review Assistant",
            "description": "TODO: Add description",
            "category": "development",
            "template": """TODO: Design your code review template
            
            Should include:
            - Code quality assessment
            - Security considerations
            - Performance suggestions
            - Best practice recommendations
            """,
            "variables": ["code", "language", "review_focus"],
            "examples": [
                # TODO: Add example usage
            ],
            "tags": ["code_review", "development"]
        },
        # TODO: Add more custom templates
    ]
    
    # TODO: Add templates to library
    for template_data in custom_templates:
        # library.add_template(template_data)
        pass
    
    console.print(f"[green]Added {len(custom_templates)} custom templates to library[/green]")

# TODO: Run the function to add your templates
# create_custom_templates()

print("🎨 Custom template creation ready!")
print("⚠️ TODO: Design and implement your custom prompt templates")

## 🎯 Exit Ticket

Before completing this lab, make sure you can answer:

### ✅ Deliverables Checklist

- [ ] **Built template management system**: Add, store, and retrieve prompt templates
- [ ] **Implemented template browser**: Search and discover templates by category/tags
- [ ] **Created testing framework**: Validate templates with examples and benchmarks
- [ ] **Added 3+ custom templates**: Templates for different domains/use cases
- [ ] **Demonstrated end-to-end workflow**: From template creation to testing and usage

### 🧠 Knowledge Check

1. **How do you ensure template quality?** What validation should you implement?

2. **When would you version a template?** What changes require a new version?

3. **How do you organize templates for a team?** What metadata is most important?

4. **What makes a good template?** Balance between flexibility and specificity?

### 🚀 Extensions (Optional)

- **Template inheritance**: Base templates that others can extend
- **A/B testing**: Compare template versions automatically
- **Performance analytics**: Track usage patterns and success rates
- **Team collaboration**: Sharing and reviewing templates
- **Template marketplace**: Import templates from external sources

### 📊 Success Metrics

- Built working prompt library with 5+ templates
- Implemented search and browsing capabilities
- Created validation and testing framework
- Designed templates covering multiple domains
- Demonstrated template reuse and customization

**Time Check:** This lab should take about 60 minutes. Focus on getting core functionality working before adding advanced features.

Ready for Lab 7: Structured Output? Let's build reliable JSON pipelines! 🔧